# Hecho servicios por hora

In [1]:
import datetime
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine

# Añadir conexión a la base de datos

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Extracción de datos

In [4]:
servicios_df = pd.read_sql("SELECT * FROM mensajeria_servicio", ryf_conn)
hora_df = pd.read_sql("SELECT * FROM dim_hora", etl_conn)

# Transformación

In [5]:
servicios_por_hora = servicios_df.groupby(['hora_solicitud']).size().reset_index(name='total_servicios')

# Carga al datamart

In [6]:
servicios_por_hora.to_sql("servicios_por_hora", etl_conn, if_exists="replace", index=False)

178